In [1]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

from urllib.request import urlopen
import requests
import pandas as pd
from json import dumps

from resource_consolidation import ResourceOperations

In [2]:
url = 'https://data.wprdc.org/dataset/healthyride-trip-data'
resource_extension = '/dataset/healthyride-trip-data/resource/'
resource_calling_string = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id='


In [3]:
list_of_links = ResourceOperations.list_of_links(url)
overall = ResourceOperations.extract_unique_path(list_of_links, resource_extension, resource_calling_string)

In [4]:
json_list = ResourceOperations.combine_json(overall)
jsonStr = dumps(json_list)

working on: https://data.wprdc.org/api/3/action/datastore_search?resource_id=c2608a60-9282-4e3d-b45d-a0306b75a3c9
of length: 11322
working on: https://data.wprdc.org/api/3/action/datastore_search?resource_id=102b0be5-aa3f-48b5-9306-e628ae1ae44d
of length: 7224
working on: https://data.wprdc.org/api/3/action/datastore_search?resource_id=6ab85d5c-b7a1-41f2-a7a5-1eec8cea4d8c
of length: 19855
working on: https://data.wprdc.org/api/3/action/datastore_search?resource_id=f8e0d66b-3958-4c8c-97d0-4351b9dfb222
of length: 5186
working on: https://data.wprdc.org/api/3/action/datastore_search?resource_id=79fc7302-7991-4b5a-b27a-d7c4f21b5b8d
of length: 12225
working on: https://data.wprdc.org/api/3/action/datastore_search?resource_id=2c29e217-935d-4d66-909f-9f1b060ed36c
of length: 24313
working on: https://data.wprdc.org/api/3/action/datastore_search?resource_id=2617daa2-4f89-4e2c-9be9-98254f1d8352
of length: 28841
working on: https://data.wprdc.org/api/3/action/datastore_search?resource_id=fc3bf582

In [5]:
df = pd.read_json(jsonStr)
#double checking head of dataframe to see if it correctly combined
print(df.head(n=10))

      Trip id   Bikeid                                    To station name  \
0  24420718.0  70233.0                                 21st St & Penn Ave   
1  24420723.0  70103.0                                 21st St & Penn Ave   
2  24433623.0  70255.0                          Penn Ave & S Whitfield St   
3  24420960.0  70197.0                       Ellsworth Ave & N Neville St   
4  24421083.0  70372.0                            S 12th St & E Carson St   
5  24421586.0  70126.0                              Ivy St & Walnut St      
6  24423420.0  70059.0                Isabella St & Federal St (PNC Park)   
7  24425934.0  70365.0  O'Hara St and University Place (Soldiers and S...   
8  24428018.0  70317.0                              S 18th St & Sidney St   
9  24428718.0  70262.0                           Liberty Ave & Stanwix St   

     Usertype        Stoptime  \
0    Customer  10/1/2016 2:01   
1  Subscriber  10/1/2016 2:00   
2    Customer  10/1/2016 9:21   
3    Customer  10/1/

In [6]:
#saving copy of dataframe in original state
checkpoint_1_df = df.copy()